In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pickle
from astropy.table import Table

if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
from pyutils import *
from dataloc import *
import groupcatalog as gc
from nnanalysis import *
import catalog_definitions as cat
from redshift_guesser import SimpleRedshiftGuesser, PhotometricRedshiftGuesser
from groupcatalog import TestGroupCatalog, BGSGroupCatalog

%load_ext autoreload
%autoreload 2

In [2]:
blue_color = GLOBAL_RED_COLOR_CUT - 0.1
red_color = GLOBAL_RED_COLOR_CUT + 0.1

blue_dn = -1
red_dn = 3

results = is_quiescent_BGS_smart(np.array([7,8,9]), np.array([red_dn, np.nan, blue_dn]), np.array([blue_color, blue_color, red_color]))
assert results[0] == True
assert results[1] == False
assert results[2] == False

In [ ]:
# Basic test of multiple versions of SimpleRedshiftGuesser
# Ensure it handles arrays of inputs and gives a reasonable answer for a couple obvious cases

# Target (lost galaxies) properties
t_app_mag = np.array([19.0,18.0,12.0])
t_pobs = np.array([0.5, 0.5, 0.5])
t_q = np.array([True, True, False])

# Neighbor properties
nn_z = np.array([0.1, 0.2, 0.3])
nn_dist = np.array([250.0, 3.0, 30.0])
nn_q = np.array([True, True, False])

simple = SimpleRedshiftGuesser(None, None, ver='5.0')
z, assignment_type = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not assignment_type[0]
assert assignment_type[1]
assert not assignment_type[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1

simple = SimpleRedshiftGuesser(None, None, ver='4.0')
z, assignment_type = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not assignment_type[0]
assert assignment_type[1]
assert not assignment_type[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1, z[2]

simple = SimpleRedshiftGuesser(None, None, ver='2.0')
z, assignment_type = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not assignment_type[0]
assert assignment_type[1]
assert not assignment_type[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1


In [ ]:
# Basic test on scores from the SV3 bins file 
target_prob_obs = None #[0.5]
target_app_mag = [19.0, 19.0, 18.0, 14.0]
target_quiescent = [1.0, 0.0, 1.0, 1.0]
neighbor_z = [0.1, 0.2, 0.05, 0.3]
neighbor_ang_dist = [30.0, 150.0, 2.0, 15.0]
nn_quiescent = [1.0, 0.0, 1.0, 1.0]

nna = NNAnalyzer_cic.from_results_file(NEIGHBOR_ANALYSIS_SV3_BINS_SMOOTHED_FILE)
score_b = nna.get_score(target_prob_obs, target_app_mag, target_quiescent, neighbor_z, neighbor_ang_dist, nn_quiescent)
print(score_b)

assert score_b[0] > 0.01, f"Reasonable parameters should have a non-zero score, but got {score_b[0]}"
#assert score_b[1] < 0.1, f"Very high angular distance should have a low score, but got {score_b[1]}" # BUG is it that issue I saw where edge value is True and spreads?
assert score_b[2] > 0.4, f"Very low angular distance should have a high score, but got {score_b[2]}"
assert score_b[3] < 0.1, f"Bright target with neighbor at high z should have a low score even at close distance {score_b[3]}"


In [ ]:
# Basic tests for PhotometricRedshiftGuesser

# Target (lost galaxies) properties
t_app_mag = np.array([19.0,18.0,12.0])
t_pobs = np.array([0.5, 0.5, 0.5])
t_q = np.array([True, True, False])
t_zphot = np.array([0.105, 0.230, 0.011])

# Neighbor properties
nn_z = np.array([[0.1, 0.2, 0.3],
                 [0.2, 0.3, 0.1]])
nn_dist = np.array([[250.0, 3.0,  30.0],
                    [260.0, 40.0, 40.0]])
nn_q = np.array([[True, True, False],
                 [False, False, False]])

scorer = PhotometricRedshiftGuesser.from_files(BGS_Y3_LOST_APP_TO_Z_FILE, NEIGHBOR_ANALYSIS_SV3_BINS_SMOOTHED_FILE, Mode.PHOTOZ_PLUS_v1)
params = ((0.8, 1.0, 3),(0.8, 1.0, 3),(0.8, 1.0, 3),(0.8, 1.0, 3))
z, assignment_type = scorer.choose_redshift(nn_z, nn_dist, t_zphot, t_pobs, t_app_mag, t_q, nn_q, params)

print(z)
print(assignment_type)

# TODO more tests

assert assignment_type[0] == AssignedRedshiftFlag.PSEUDO_RANDOM.value
assert assignment_type[1] == 1, "should use neighbor 1 "
assert assignment_type[2] == AssignedRedshiftFlag.PSEUDO_RANDOM.value

assert np.isnan(z).sum() == 0, "z should not have any NaNs"
assert z[0] > 0.0, "z[0] should be greater than 0.0"
assert z[1] == 0.2, "z[1] should be 0.2"
assert z[2] < 0.1, "z[2] should be less than 0.1"

In [ ]:
# Test out sim_z_score function
# Test out smooth redshift comparison function works as desired for the relevant redshift differences
x=np.arange(-0.100, 0.105, 0.0005)
plt.plot(x, sim_z_score(0.2, 0.2+x))
plt.plot(x, close_enough_smooth(0.2, 0.2-x))
plt.axvline(0.005, color='g')
plt.axvline(-0.005, color='g')
plt.tight_layout()
plt.xlabel("Redshift difference")
plt.ylabel("Similarity score")

# Make sure the extremes are working as expected
assert sim_z_score(0.1, 0.3) < 0.01
assert sim_z_score(0.2, 0.3) < 0.05 and sim_z_score(0.2, 0.3) > 0.01
assert sim_z_score(0.2, 0.25) < 0.1 and sim_z_score(0.2, 0.25) > 0.05
assert sim_z_score(0.2, 0.210) > 0.2, sim_z_score(0.2, 0.210)
assert sim_z_score(0.2, 0.205) > 0.95, sim_z_score(0.2, 0.205)
assert sim_z_score(0.2, 0.203) > 0.99, sim_z_score(0.2, 0.203)
assert np.isclose(sim_z_score(0.2, 0.2001), 1.0)


### CIC Binning

In [ ]:
# 2D test of my N-dimensional CIC binning function
data_2d = np.array([
    [0.0, 0.0], # Test corner case
    [0.0, -0.5], # Test left edge case
    [3.5, 3.5], # Test middle case
    [2, 5.9],
    [0.5, 40.0], # Test right edge case
    [-7.0, -3.0], # Extra edge case
])
first_dim  = np.linspace(0, 5, 6)
second_dim  = np.linspace(0, 6, 7)

bin_counts = cic_binning(data_2d, [first_dim, second_dim])
print(bin_counts)

assert np.shape(bin_counts) == (6, 7), np.shape(bin_counts)
assert np.sum(bin_counts) == len(data_2d), np.sum(bin_counts)
assert np.isclose(bin_counts[0,0], 3.0), bin_counts[0,0]
assert np.isclose(bin_counts[2,5], 0.1), bin_counts[2,5]
assert np.isclose(bin_counts[2,6], 0.9), bin_counts[2,6]
assert np.isclose(bin_counts[3,3], 0.25), bin_counts[3,3]
assert np.isclose(bin_counts[3,4], 0.25), bin_counts[3,4]
assert np.isclose(bin_counts[4,3], 0.25), bin_counts[4,3]
assert np.isclose(bin_counts[4,4], 0.25), bin_counts[4,4]
assert np.isclose(bin_counts[4,4], 0.25), bin_counts[4,4]
assert np.isclose(bin_counts[0,6], 0.5), bin_counts[0,6]
assert np.isclose(bin_counts[1,6], 0.5), bin_counts[1,]

In [ ]:
# 3D test of CIC binning
data_3d = np.array([
    [0.0, 0.0, 0.0],  # Test corner case
    [1.5, 1.5, 1.5],  # Test middle case
    [10.0, 10.0, -10.0],  #  edge case
    [0.0, -1.0, 1.6]
])
first_dim_3d = np.linspace(0, 3, 4)
second_dim_3d = np.linspace(0, 4, 5)
third_dim_3d = np.linspace(0, 2, 3)

# Perform CIC binning
bin_counts_3d = cic_binning(data_3d, [first_dim_3d, second_dim_3d, third_dim_3d])
print(bin_counts_3d)

# Assertions to verify the binning results
assert np.shape(bin_counts_3d) == (4, 5, 3), np.shape(bin_counts_3d)
assert np.isclose(np.sum(bin_counts_3d), len(data_3d)), np.sum(bin_counts_3d)
assert np.isclose(bin_counts_3d[0,0,0], 1.0), bin_counts_3d[0,0,0]
assert np.isclose(bin_counts_3d[1,1,1], 1/8), bin_counts_3d[1,1,1]
assert np.isclose(bin_counts_3d[1,1,2], 1/8), bin_counts_3d[1,1,2]
assert np.isclose(bin_counts_3d[1,2,1], 1/8), bin_counts_3d[1,2,1]
assert np.isclose(bin_counts_3d[1,2,2], 1/8), bin_counts_3d[1,2,2]
assert np.isclose(bin_counts_3d[2,1,1], 1/8), bin_counts_3d[2,1,1]
assert np.isclose(bin_counts_3d[2,1,2], 1/8), bin_counts_3d[2,1,2]
assert np.isclose(bin_counts_3d[2,2,1], 1/8), bin_counts_3d[2,2,1]
assert np.isclose(bin_counts_3d[2,2,2], 1/8), bin_counts_3d[2,2,2]
assert np.isclose(bin_counts_3d[3,4,0], 1.0), bin_counts_3d[3,4,0]
assert np.isclose(bin_counts_3d[0,0,1], 0.4), bin_counts_3d[3,4,1]
assert np.isclose(bin_counts_3d[0,0,2], 0.6), bin_counts_3d[3,4,1]

In [ ]:
# Test CIC binning with weights
data_2d = np.array([
    [0.0, 0.0],
    [0.0, 1.0],
])
first_dim  = np.linspace(0, 1, 2)
second_dim  = np.linspace(0, 1, 2)

bin_counts = cic_binning(data_2d, [first_dim, second_dim], weights=[0.66, 2.99])
print(bin_counts)

assert np.shape(bin_counts) == (2, 2), np.shape(bin_counts)
assert np.isclose(bin_counts[0,0], 0.66), bin_counts[0,0]
assert np.isclose(bin_counts[0,1], 2.99), bin_counts[0,0]

In [ ]:
# Test CIC binning with repeats
data_2d = np.array([
    [0.0, 0.0],  
    [1.0, 1.0],  
    [1.0, 1.0],  
    [1.0, 1.0],  
])
first_dim  = np.linspace(0, 1, 2)
second_dim  = np.linspace(0, 1, 2)

bin_counts = cic_binning(data_2d, [first_dim, second_dim])
print(bin_counts)

assert np.shape(bin_counts) == (2, 2), np.shape(bin_counts)
assert np.isclose(bin_counts[0,0], 1.0), bin_counts[0,0]
assert np.isclose(bin_counts[0,1], 0.0), bin_counts[0,1]
assert np.isclose(bin_counts[1,0], 0.0), bin_counts[1,0]
assert np.isclose(bin_counts[1,1], 3.0), bin_counts[1,1]

In [ ]:
# Test CIC binning with negative dimensional ranges
data_2d = np.array([
    [-1.0, -1.2],  
    [0.5, 1.0],  
])
first_dim  = np.linspace(-1, 1, 3)
second_dim  = np.linspace(-1, 1, 3)

bin_counts = cic_binning(data_2d, [first_dim, second_dim])
print(bin_counts)

assert np.shape(bin_counts) == (3, 3), np.shape(bin_counts)
assert np.isclose(np.sum(bin_counts), 2.0), bin_counts[0,0]
assert np.isclose(bin_counts[0,0], 1.0), bin_counts[0,0]
assert np.isclose(bin_counts[1,2], 0.5), bin_counts[1,1]
assert np.isclose(bin_counts[2,2], 0.5), bin_counts[1,1]

In [ ]:
# Stress test of CIC binning
data_stress = np.random.rand(1000000, 5) # 5M rows of random data
dim_stress = np.linspace(0, 1, 11)

# Perform CIC binning
bin_counts_stress = cic_binning(data_stress, [dim_stress, dim_stress, dim_stress, dim_stress, dim_stress])   

# Assertions to verify the binning results
assert np.shape(bin_counts_stress) == (11, 11, 11, 11, 11), np.shape(bin_counts_stress)
assert np.isclose(np.sum(bin_counts_stress), len(data_stress)), np.sum(bin_counts_stress)

with np.printoptions(precision=1, suppress=True, linewidth=100):
    print(np.sum(np.sum(np.sum(bin_counts_stress, axis=0), axis=0), axis=0))

In [ ]:
# Test CIC binning with negative dimensional ranges
data_2d = np.array([
    [-0.01, APP_MAG_BINS[3]],  
    [0.01, APP_MAG_BINS[3]],  
    [0.03, APP_MAG_BINS[3]],  
    [0.12, APP_MAG_BINS[3]],  
    [0.21, APP_MAG_BINS[3]],  
    [0.40, APP_MAG_BINS[3]],  
    [0.50, APP_MAG_BINS[5]],  
])
first_dim  = Z_BINS
second_dim  = APP_MAG_BINS

bin_counts = cic_binning(data_2d, [first_dim, second_dim])

with np.printoptions(precision=3, suppress=True, linewidth=200):
    print(bin_counts)



### Other

In [ ]:
# TODO not sure what this was
"""
with open('SimpleRedshiftGuesserMap.pkl', 'rb') as f:    
    app_mag_bins, the_map = pickle.load(f)

print(the_map.keys())

indexes = [10,11,30,45]
# histogram of the map at those indexes
for i in indexes:
    plt.hist(the_map[i], bins = 20)
    plt.title(f'app mag ~ {app_mag_bins[i-1]}')
    plt.show()

test_mags = np.linspace(12.0, 20.0, 10000)
test_z = np.linspace(0.0, 0.5, 10000) * np.random.rand(10000)
app_mag_bins, the_map = build_app_mag_to_z_map_new(test_mags, test_z)

print(the_map)
"""

In [ ]:
# Question: which is faster, boolean indexing or integer indexing?
arr = np.random.rand(20000000)
bool_filter = arr < 0.1
N = 20

test = np.zeros(bool_filter.sum())
test2 = np.zeros(len(arr))
t1 = time.time()
for n in range(N):
    test += arr[bool_filter]
    test2[bool_filter] += arr[bool_filter]
t2 = time.time()
print(f"Time for bool filter: {t2-t1}")

test = np.zeros(bool_filter.sum())
t1 = time.time()
idx_filter = np.flatnonzero(bool_filter)
for n in range(N):
    test += arr[idx_filter]
    test2[idx_filter] += arr[idx_filter]
t2 = time.time()
print(f"Time for idx filter: {t2-t1}")

# Answer: idx filter is faster. And order doesn't matter

In [ ]:
import time

# function signature is write_dat_files(ra, dec, z_eff, log_L_gal, V_max, colors, chi, outname_base, frac_area):

# Generate some sample data to write
NUM_ROWS = 500000
ra = np.random.rand(NUM_ROWS)
dec = np.random.rand(NUM_ROWS)
z_eff = np.random.rand(NUM_ROWS)
log_L_gal = np.random.rand(NUM_ROWS)
V_max = np.random.rand(NUM_ROWS)
colors = np.random.randint(0, 2, NUM_ROWS)
chi = np.random.rand(NUM_ROWS)
outname_base = OUTPUT_FOLDER + 'speed-write-test'
outname_base2 = OUTPUT_FOLDER + 'speed-write-test2'

# Write data using write_dat_files
start_time = time.time()
write_dat_files(ra, dec, z_eff, log_L_gal, V_max, colors, chi, outname_base)
end_time = time.time()
time_v1 = end_time - start_time

# Write data using write_dat_files_v2
start_time = time.time()
write_dat_files_v2(ra, dec, z_eff, log_L_gal, V_max, colors, chi, outname_base2)
end_time = time.time()
time_v2 = end_time - start_time

# Compare the contents of the two files
with open(outname_base + '.dat', 'rb') as f1, open(outname_base2 + '.dat', 'rb') as f2:
    data_v1 = f1.read()
    data_v2 = f2.read()

assert data_v1 == data_v2, "The outputs of write_dat_files and write_dat_files_v2 are not the same"

# Print the time taken by each function
print(f"Time taken by write_dat_files: {time_v1} seconds")
print(f"Time taken by write_dat_files_v2: {time_v2} seconds")

In [ ]:
dataMat1 = np.random.rand(1000,1000)
dataMat2 = np.random.rand(2,500000)
dataMat3 = np.random.rand(500000,2)
start = time.perf_counter()
with open(OUTPUT_FOLDER + 'test1.txt','w') as f:
    np.savetxt(f,dataMat1,fmt='%g',delimiter=' ')
end = time.perf_counter()
print(end-start)

start = time.perf_counter()
with open(OUTPUT_FOLDER +  'test2.txt','w') as f:
    np.savetxt(f,dataMat2,fmt='%g',delimiter=' ')
end = time.perf_counter()
print(end-start)

start = time.perf_counter()
with open(OUTPUT_FOLDER +  'test3.txt','w') as f:
    np.savetxt(f,dataMat3,fmt='%g',delimiter=' ')
end = time.perf_counter()
print(end-start)

start = time.perf_counter()
with open(OUTPUT_FOLDER + 'test4.txt','w') as f:
    fmt = ' '.join(['%g']*dataMat3.shape[1])
    fmt = '\n'.join([fmt]*dataMat3.shape[0])
    data = fmt % tuple(dataMat3.ravel())        
    f.write(data)
end = time.perf_counter()
print(end-start)

# End to End Test

In [25]:
# Only needs to be run once, unless you want to change the test data
#catalog = TestGroupCatalog("Test")
#catalog.create_test_dat_files() 

In [ ]:
SV3_test = BGSGroupCatalog("SV3 Test", Mode.SIMPLE_v4, 19.5, 21.0, num_passes=10, drop_passes=3, data_cut='sv3', sdss_fill=False)
SV3_test.GF_props = cat.GF_PROPS_VANILLA.copy()

SV3_test.preprocess()

# Read in BGS_SV3_ANY_FULL_FILE and ensure no precision is lost from there to SV3_test.preprocess_file and the like
merged_table = Table.read(IAN_BGS_SV3_MERGED_FILE, format='fits')
print(merged_table['RA'][0:10])

# read in and print out the first few lines of SV3_test.preprocess_file
with open(SV3_test.preprocess_file, 'r') as f:
    for i in range(10):
        print(f.readline(), end='')

#with open(SV3_test.preprocess_file + "~", 'r') as f:
#    for i in range(10):
#        print(f.readline(), end='')

galprops_file = str.replace(SV3_test.GF_outfile, ".out", "_galprops.dat")
with open(galprops_file, 'r') as f:
    for i in range(10):
        print(f.readline(), end='')

#with open(galprops_file + "~", 'r') as f:
#    for i in range(10):
#        print(f.readline(), end='')


In [ ]:
# C Group Finder Tests



np.set_printoptions(threshold=sys.maxsize)

# Baseline vanilla group finder test 
catalog = TestGroupCatalog("Test")
catalog.run_group_finder(silent=True) 
catalog.postprocess()
df=catalog.all_data
baseline_total_mass = df['M_halo'].sum()
assert len(np.unique(df['igrp'])) == 200
assert len(df) == 246 
assert df['quiescent'].sum() == 129
assert np.isclose(df['weight'].sum(), 246 * 1.0) # no weights, just 1 per gal
m1=df['M_halo'].to_numpy()

# Test that when omega0 are 0, the others don't matter
catalog = TestGroupCatalog("Test")
catalog.GF_props['omegaL_sf'] = 123
catalog.GF_props['sigma_sf'] = 345
catalog.GF_props['omegaL_q'] = 456
catalog.GF_props['sigma_q'] = 678
catalog.GF_props['omega0_sf'] = 0.0
catalog.GF_props['omega0_q'] = 0.0
catalog.run_group_finder(silent=True)
catalog.postprocess()
df=catalog.all_data
assert len(np.unique(df['igrp'])) == 200
assert len(df) == 246 
assert df['quiescent'].sum() == 129
assert np.isclose(df['weight'].sum(), 246 * 1.0) # no weights, just 1 per gal
assert np.isclose(df['M_halo'].sum(), baseline_total_mass)
m2=df['M_halo'].to_numpy()

catalog = TestGroupCatalog("Test")
#catalog.GF_props['colors'] = 1
catalog.GF_props['omegaL_sf'] = 10.0
catalog.GF_props['sigma_sf'] = 3.0
catalog.GF_props['omegaL_q'] = 0.0
catalog.GF_props['sigma_q'] = 0.0
catalog.GF_props['omega0_sf'] = 10.0
catalog.GF_props['omega0_q'] = 0.0
catalog.run_group_finder(silent=True)
catalog.postprocess()
df=catalog.all_data
assert len(np.unique(df['igrp'])) >= 200 # these parameters make assigned halos smaller
assert len(df) == 246 
assert df['quiescent'].sum() == 129
assert df['weight'].sum() < 246 
# TODO BUG I feel like this should be true, but it's not. Weighting doesn't preseve the halo mass function
#assert np.isclose(df['M_halo'].sum(), baseline_total_mass) 
m3=df['M_halo'].to_numpy()

plt.hist(np.stack([np.log10(m1), np.log10(m2), np.log10(m3)], axis=-1))


print("All tests passed")

In [ ]:
import plotting as pp
pp.examine_area(np.min(df.RA), np.max(df.RA), np.min(df.Dec), np.max(df.Dec), df)


# MCMC of photo-z-plus results

In [ ]:
import emcee
import groupcatalog as gc
import math
#backend = emcee.backends.HDFBackend("mcmc17_1_4.h5", read_only=True)
#backend = emcee.backends.HDFBackend("mcmc17b_1_5.h5", read_only=True)
#backend = emcee.backends.HDFBackend("mcmc17_2_0.h5", read_only=True)
#backend = emcee.backends.HDFBackend("mcmc13_2_1.h5", read_only=True)

#backend = emcee.backends.HDFBackend("SelfCalGroupFinder/py/mcmc13_2_2.h5", read_only=True)
#backend = emcee.backends.HDFBackend("SelfCalGroupFinder/py/mcmc13_1_6.h5", read_only=True)

backend = emcee.backends.HDFBackend("SelfCalGroupFinder/py/mcmc13_2_3.h5", read_only=True)
print(backend.shape)

sampler = emcee.EnsembleSampler(backend.shape[0], backend.shape[1], gc.log_probability, backend=backend)
samples = sampler.get_chain(flat=True)
print(f"Flat iterations run {len(samples)}")

# Get the log probabilities and sort them to find the top N parameter sets
log_prob = sampler.get_log_prob(flat=True)
top_N = 5  # Number of top parameter sets to display
top_indices = np.argsort(log_prob)[::-1]  # Sort in descending order

selected_indices = []
for idx in top_indices:
    params = samples[idx]
    if all(not np.all(np.isclose(params, samples[prev_idx], rtol=0.5)) for prev_idx in selected_indices):
        selected_indices.append(idx)
        if len(selected_indices) >= top_N:
            break

with np.printoptions(precision=4, suppress=True, linewidth=200):
    for i, idx in enumerate(selected_indices):
        params = samples[idx]
        print(f"Rank {i+1}:")
        print(f"Parameters: N={int(params[0])} {params[1:4]} {params[4:7]} {params[7:10]} {params[10:13]}")
        print(f"SCORE: {log_prob[idx]:.4f}")
        print()


In [ ]:
import corner

try:
    tau = sampler.get_autocorr_time()
    print(tau)
except:
    print("Not burnt in yet")

flatchain = sampler.get_chain(discard=1000, flat=True)
print(np.shape(flatchain))
fig = corner.corner(flatchain)

In [24]:
c = sampler.get_chain(discard=0)
print(np.shape(c))

for i in range(c.shape[2]):
    plt.figure()
    plt.plot(c[:,:,i])
    plt.show()
